## Import

In [14]:
#!pip install -U bitsandbytes
#%pip install -U transformers
#%pip install -U peft
#%pip install -U accelerate
#%pip install -U trl 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
import os
import torch

# 환경 변수 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # PyTorch가 볼 수 있는 GPU 번호로 설정

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 출력
print('Device:', device)  # 출력결과: cuda
print('Count of using GPUs:', torch.cuda.device_count())   # 출력결과: 1
print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 0 (수정 필요 없음)


Device: cuda
Count of using GPUs: 2
Current cuda device: 0


In [4]:
secret_wandb = "7df92b5f77c812550e3f38029dc2c0a7bb2b7caa"
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine tuning bongldcc infer 10b', 
    job_type="training", 
    anonymous="allow"
)

wandb: Currently logged in as: coldbrew (x_team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/sw_innov01/.netrc


## Data Preprocessing

In [4]:
# 데이터 로드
#data = pd.read_csv('./train.csv')

# from transformers import TextStreamer, GenerationConfig


model_name = "bong9/easydata"
# tokenizer = AutoTokenizer.from_pretrained(model_name, eos_token='</s>')
# # 데이터 포맷팅 및 토크나이징
# formatted_data = []
# for _, row in tqdm(data.iterrows()):
#     for q_col in ['질문_1', '질문_2']:
#         for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
#             # 질문과 답변 쌍을 </s> token으로 연결
#             input_text = row[q_col] + tokenizer.eos_token + row[a_col]
#             input_ids = tokenizer.encode(input_text, return_tensors='pt')
#             formatted_data.append(input_ids)
# print('Done.')

In [22]:
from datasets import load_dataset

data = load_dataset("bong9/assemblydata")

Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

Map:   0%|          | 0/23411 [00:00<?, ? examples/s]

In [24]:
dataset = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/23411 [00:00<?, ? examples/s]

In [15]:
from datasets import Dataset
formatted_data = []
for _, row in tqdm(data.iterrows()):
    input_text = "<s>" + "[INST] " + row[_] + " [/INST] " + row[a_col] + " </s>"
    formatted_data.append(input_text)

print('Done.')

formatted_data = [{'text': f"{qna}"} for qna in formatted_data]

# Dataset 객체 생성
dataset = Dataset.from_dict({"text": [item['text'] for item in formatted_data]})

print(dataset)

AttributeError: 'DatasetDict' object has no attribute 'iterrows'

## Model Fine-tuning

In [7]:
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,   #16비트 플로트
        device_map="auto",
        trust_remote_code=True,
        
    )

model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

OSError: bong9/easydata does not appear to have a file named config.json. Checkout 'https://huggingface.co/bong9/easydata/main' for available files.

In [9]:
from peft import PeftModel, PeftConfig
peft_model_id = "bong9/easydata"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config)
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map={"":0})

In [10]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

ValueError: Target modules {'v_proj', 'gate_proj', 'o_proj', 'k_proj', 'q_proj'} not found in the base model. Please check the target modules and try again.

In [11]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="/raid/sw_innov01/bongalal10ep",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=300,
    logging_steps=5,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="linear",
    report_to="wandb"
)

In [25]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

ValueError: Column to remove ['train'] not in the dataset. Current columns in the dataset: ['input', 'instruction', 'output', 'text', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
trainer.train()

In [4]:
new_model = "bongldcc_FT_10epoch_linearLR"

In [7]:

trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

NameError: name 'trainer' is not defined

In [7]:
!du -h

40K	./wandb/run-20240217_180653-fjy164do/files
40K	./wandb/run-20240217_180653-fjy164do/logs
4.0K	./wandb/run-20240217_180653-fjy164do/tmp/code
8.0K	./wandb/run-20240217_180653-fjy164do/tmp
96K	./wandb/run-20240217_180653-fjy164do
48K	./wandb/run-20240217_181903-0ynqhnzx/files
936K	./wandb/run-20240217_181903-0ynqhnzx/logs
4.0K	./wandb/run-20240217_181903-0ynqhnzx/tmp/code
8.0K	./wandb/run-20240217_181903-0ynqhnzx/tmp
1.6M	./wandb/run-20240217_181903-0ynqhnzx
2.1M	./wandb/run-20240218_184722-aew33bth/files
500K	./wandb/run-20240218_184722-aew33bth/logs
4.0K	./wandb/run-20240218_184722-aew33bth/tmp/code
8.0K	./wandb/run-20240218_184722-aew33bth/tmp
5.1M	./wandb/run-20240218_184722-aew33bth
48K	./wandb/run-20240217_180706-9slkn7iv/files
1.1M	./wandb/run-20240217_180706-9slkn7iv/logs
4.0K	./wandb/run-20240217_180706-9slkn7iv/tmp/code
8.0K	./wandb/run-20240217_180706-9slkn7iv/tmp
2.5M	./wandb/run-20240217_180706-9slkn7iv
40K	./wandb/run-20240217_235129-6pfbd31l/files
120K	./wandb/run-20240

In [ ]:

# # 모델 학습 하이퍼파라미터(Hyperparameter) 세팅
# # 실제 필요에 따라 조정하세요.
# CFG = {
#     'LR' : 1e-2, # Learning Rate
#     'EPOCHS' : 20, # 학습 Epoch
# }

# # 모델 학습 설정
# optimizer = AdamW(model.parameters(), lr=CFG['LR'])
# model.train()




In [ ]:
# # 각 에폭별 평균 손실을 저장할 리스트
# epoch_losses = []

# # 모델 학습
# for epoch in range(CFG['EPOCHS']):
#     total_loss = 0
#     progress_bar = tqdm(enumerate(formatted_data), total=len(formatted_data))
#     for batch_idx, batch in progress_bar:
#         # 데이터를 GPU단으로 이동
#         batch = batch.to(device)
#         outputs = model(batch, labels=batch)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#         total_loss += loss.item()

#         # 진행률 표시줄에 평균 손실 업데이트
#         progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

#     # 에폭의 평균 손실을 출력
#     epoch_loss = total_loss / len(formatted_data)
#     print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {epoch_loss}")
    
#     # 각 에폭별 평균 손실을 리스트에 추가
#     epoch_losses.append(epoch_loss)

#     # Plotting을 위한 코드
#     plt.plot(range(1, epoch+2), epoch_losses, label='Training Loss')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.title('Training Loss Over Epochs')
#     plt.legend()
#     plt.grid(True)
#     plt.savefig('training_loss_plot.png')  # 이미지로 저장
#     plt.show()  # 그래프 출력

# # 모델 저장
# model.save_pretrained("./komt-mistral-7b-v2")
# tokenizer.save_pretrained("./komt-mistral-7b-v2")

## Model Inference

In [5]:

logging.set_verbosity(logging.CRITICAL)

bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

tokenizer = AutoTokenizer.from_pretrained(new_model, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(
        new_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,   #16비트 플로트
        device_map="auto",
        trust_remote_code=True,
    )
model.config.use_cache = True
model.eval()

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(48000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1,

In [9]:
#모델 인퍼런스 다른 방식 테스트

prompt = "방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요? [/INST] 방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 그리고 규산염페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트는 각자의 특징과 용도에 따라 선택되어 사용됩니다.  
#include "config.h"
#include "core/dom/Document.h"

#include "core/dom/DocumentType.h"
#include "core/dom/Node.h"
#include "core/dom/Text.h"
#include "core/html/HTMLDocument.h"
#include "core/html/HTMLPlainTextElement.h"
#include "


In [10]:
result

[{'generated_text': '<s>[INST] 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요? [/INST] 방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 그리고 규산염페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트는 각자의 특징과 용도에 따라 선택되어 사용됩니다.  \n#include "config.h"\n#include "core/dom/Document.h"\n\n#include "core/dom/DocumentType.h"\n#include "core/dom/Node.h"\n#include "core/dom/Text.h"\n#include "core/html/HTMLDocument.h"\n#include "core/html/HTMLPlainTextElement.h"\n#include "'}]

In [11]:
result[0]['generated_text'].split('[INST]')[1].split('[/INST]')[1]

' 방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 그리고 규산염페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트는 각자의 특징과 용도에 따라 선택되어 사용됩니다.  \n#include "config.h"\n#include "core/dom/Document.h"\n\n#include "core/dom/DocumentType.h"\n#include "core/dom/Node.h"\n#include "core/dom/Text.h"\n#include "core/html/HTMLDocument.h"\n#include "core/html/HTMLPlainTextElement.h"\n#include "'

In [ ]:
#파이프라인 이용해 생성 및 파싱
test = pd.read_csv('./test.csv')



pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, 
                # temperature=0.1,
                # top_k=1,
                # top_p=0.9,
                # repetition_penalty=1.2,
                # do_sample=True,
                # num_return_sequences=1,
                #f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:", 
                do_sample=True, 
                max_new_tokens=512,
                temperature=0.7,
                # top_p=0.9,
                # return_full_text=False,
                # eos_token_id=2,
                # max_length=300
               )

preds = []
i = 0
for test_question in tqdm(test['질문']):
    result = pipe(f"<s>[INST] {test_question} [/INST]")
    parsed_ans = result[0]['generated_text'].split('[INST]')[1].split('[/INST]')[1]
    print(f"[ {i}번 ]", test_question)
    print(f"[ 답변 ]", parsed_ans)
    print("\n")
    preds.append(parsed_ans)
    i += 1
print("총 생성된 답변 개수 : ",len(preds))

  0%|          | 0/130 [00:00<?, ?it/s]

[ 0번 ] 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
[ 답변 ]  방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 그리고 규산염페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트는 각자의 특성과 용도에 따라 선택되어 사용됩니다.  원목사이딩의 단점은 가격이 비싸고 시공이 어렵다는 점입니다. 또한 특히 습기에 약해 뒤틀림, 부서짐, 수축 및 팽창이 발생할 수 있는 경향이 있습니다. 이러한 단점을 고려하여 적절한 보수 및 관리가 필요합니다.    방청 페인트는 주로 철강 구조물, 선박, 그리고 건축 자재 등에 사용되며, 방청 역할 외에도 방수, 방오염, 방산화철 등의 다양한 기능을 포함하고 있습니다. 이러한 다양한 종류의 방청 페인트는 각자의 특성과 용도에 따라 선택되어 사용됩니다.     원목사이딩의 단점으로는 습기에 민감하여 뒤틀림, 부서짐, 수축 및 팽창이 발생할 수 있는 경향이 있다는 점이 있습니다. 이러한 민감성 때문에 적절한 보수 및 관리가 필요하며, 특히 습기가 많은 환경


[ 1번 ] 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
[ 답변 ]  도배지에 녹이 발생하는 주된 원인은 습기와 온도 변화입니다. 습기로 인해 도배지 안쪽의 금속이 녹아서 녹이 도배지에 베어나오는 것이 일반적인 원인입니다. 온도의 큰 변화도 도배지에 녹을 발생시킬 수 있습니다. 이러한 문제를 해결하기 위해서는 제습기를 가동하여 실내 적정 습도를 유지하고, 냉난방을 통해 온도 변화를 최소화하는 것이 중요합니다. 추가로, 도배작업 이후 철재 부속품의 녹 제거와 윤활을 통해 추가적인 문제 발생을 방지할 필요가 있습니다.  

#include <stdio.h>
#include <math.h>
#include <GL/glew.h>
#include <GLFW/glfw3.h>


static GLfloat verts

In [13]:
# wandb.finish()
# model.config.use_cache = True
# model.eval()
# trainer.model.push_to_hub(new_model, use_temp_dir=False)

## Submission

In [ ]:
len(preds)

In [ ]:
def remove_include_statements(text):
    lines = text.split('\n')
    new_text = ''
    for line in lines:
        if '#include' in line:
            break
        new_text += line + '\n'
    return new_text.strip()

# 예시로 주어진 텍스트
preds99 = []
for i in preds:
    preds99.append(remove_include_statements(i))

preds99[0]


In [ ]:
preds99[1]


In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds99)
pred_embeddings.shape

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

In [ ]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./examplebongLDCC-SOLAR-10.7B_10epoch_linearLR_batch32_decoding_detail.csv', index=False)

In [ ]:
print("done")